## Top 10 arrival airports in the world in 2013 (using the bookings file)

Arrival airport is the column arr_port. It is the IATA code for the airport

To get the total number of passengers for an airport, you can sum the column pax, grouping by arr_port.

Note that there is negative pax. That corresponds to cancelations. So to get the total number of passengers that have actually booked, you should sum including the negatives (that will remove the canceled bookings).

Print the top 10 arrival airports in the standard output, including the number of passengers.

Bonus point: Get the name of the city or airport corresponding to that airport (programatically, we suggest to have a look at GeoBases in Github)

Bonus point: Solve this problem using pandas (instead of any other approach)


## 0) Prepare booking Sample (we might need this for later)

In [1]:
!pwd

/home/dsc/Data/DS_Master_A1_Chall


In [2]:
cd ../challenge/

/home/dsc/Data/challenge


In [4]:
ls -l

total 1013840
-rw-rw-r--. 1 dsc dsc 554970628 Jan  4 15:59 bookings.csv.bz2
-rw-rw-r--. 1 dsc dsc         0 May  5 11:58 bookings.sample.csv
-rw-rw-r--. 1 dsc dsc 483188920 Jan  4 16:01 searches.csv.bz2


In [5]:
! bzcat bookings.csv.bz2 | head -10000 > bookings.sample.csv


bzcat: I/O or other error, bailing out.  Possible reason follows.
bzcat: Broken pipe
	Input file = bookings.csv.bz2, output file = (stdout)


In [6]:
! wc -l  bookings.sample.csv

10000 bookings.sample.csv


In [7]:
! bzip2 bookings.sample.csv

In [8]:
! bzcat bookings.sample.csv.bz2 | wc -l

10000


## 1) Get familiar with data

In [9]:
import pandas as pd

### What if we dont want to read the whole file?

In [22]:
b = pd.read_csv('bookings.sample.csv.bz2',sep='^')

In [10]:
b=pd.read_csv('bookings.csv.bz2',sep='^', nrows=1000)

In [ ]:
b.info()

In [47]:
b.head()

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,...,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
0,2013-03-05 00:00:00,1A,DE,a68dd7ae953c8acfb187a1af2dcbe123,1a11ae49fcbf545fd2afc1a24d88d2b7,ea65900e72d71f4626378e2ebd298267,2013-02-22 00:00:00,1708,0,ZRH,...,LHRZRH,VI,T,Y,2013-03-07 08:50:00,2013-03-07 11:33:37,-1,2013,3,NULL
1,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,...,SALATLCLT,NV,L,Y,2013-04-12 13:04:00,2013-04-12 22:05:40,1,2013,3,NULL
2,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,...,CLTATLSAL,NV,U,Y,2013-07-15 07:00:00,2013-07-15 11:34:51,1,2013,3,NULL
3,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,...,AKLHKGSVO,XK,G,Y,2013-04-24 23:59:00,2013-04-25 16:06:31,1,2013,3,SYDA82546
4,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,...,SVOHKGAKL,XK,G,Y,2013-05-14 20:15:00,2013-05-16 10:44:50,1,2013,3,SYDA82546


In [70]:
b.columns

Index([u'act_date           ', u'source', u'pos_ctry', u'pos_iata',
       u'pos_oid  ', u'rloc          ', u'cre_date           ', u'duration',
       u'distance', u'dep_port', u'dep_city', u'dep_ctry', u'arr_port',
       u'arr_city', u'arr_ctry', u'lst_port', u'lst_city', u'lst_ctry',
       u'brd_port', u'brd_city', u'brd_ctry', u'off_port', u'off_city',
       u'off_ctry', u'mkt_port', u'mkt_city', u'mkt_ctry', u'intl',
       u'route          ', u'carrier', u'bkg_class', u'cab_class',
       u'brd_time           ', u'off_time           ', u'pax', u'year',
       u'month', u'oid      '],
      dtype='object')

In [72]:
b.columns.tolist()

['act_date           ',
 'source',
 'pos_ctry',
 'pos_iata',
 'pos_oid  ',
 'rloc          ',
 'cre_date           ',
 'duration',
 'distance',
 'dep_port',
 'dep_city',
 'dep_ctry',
 'arr_port',
 'arr_city',
 'arr_ctry',
 'lst_port',
 'lst_city',
 'lst_ctry',
 'brd_port',
 'brd_city',
 'brd_ctry',
 'off_port',
 'off_city',
 'off_ctry',
 'mkt_port',
 'mkt_city',
 'mkt_ctry',
 'intl',
 'route          ',
 'carrier',
 'bkg_class',
 'cab_class',
 'brd_time           ',
 'off_time           ',
 'pax',
 'year',
 'month',
 'oid      ']

In [25]:
list(b.columns)

['act_date           ',
 'source',
 'pos_ctry',
 'pos_iata',
 'pos_oid  ',
 'rloc          ',
 'cre_date           ',
 'duration',
 'distance',
 'dep_port',
 'dep_city',
 'dep_ctry',
 'arr_port',
 'arr_city',
 'arr_ctry',
 'lst_port',
 'lst_city',
 'lst_ctry',
 'brd_port',
 'brd_city',
 'brd_ctry',
 'off_port',
 'off_city',
 'off_ctry',
 'mkt_port',
 'mkt_city',
 'mkt_ctry',
 'intl',
 'route          ',
 'carrier',
 'bkg_class',
 'cab_class',
 'brd_time           ',
 'off_time           ',
 'pax',
 'year',
 'month',
 'oid      ']

In [27]:
b.shape

(1000, 38)

In [57]:
description=b.describe()

In [58]:
description

,duration,distance,intl,pax,year,month
count,1000.000000,1000.000000,1000.000000,1000.000000,1000,1000
mean,23935.433000,407.569000,0.643000,0.532000,2013,3
std,54459.101785,1490.087967,0.479355,1.976077,0,0
min,61.000000,0.000000,0.000000,-8.000000,2013,3
25%,3493.000000,0.000000,0.000000,-1.000000,2013,3
50%,6891.000000,0.000000,1.000000,1.000000,2013,3
75%,17940.000000,0.000000,1.000000,1.000000,2013,3
max,482174.000000,10995.000000,1.000000,21.000000,2013,3


In [73]:
b.describe(include='all')

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,dep_city,dep_ctry,arr_port,arr_city,arr_ctry,lst_port,lst_city,lst_ctry,brd_port,brd_city,brd_ctry,off_port,off_city,off_ctry,mkt_port,mkt_city,mkt_ctry,intl,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
count,1000,1000,1000,1000,1000,1000,1000,1000.000000,1000.000000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000.000000,1000,1000,1000,1000,1000,1000,1000.000000,1000,1000,1000
unique,31,5,53,278,281,352,57,NaN,NaN,176,166,57,201,195,74,184,175,60,263,250,85,267,253,83,403,395,197,NaN,634,86,26,3,688,704,NaN,NaN,NaN,45
top,2013-03-06 00:00:00,1P,US,83ccf07a55606976f78e63fcc805f58c,1490c3c5f95f04fa076fe15c775a80a9,a37584d1485cb35991e4ff1a2ba92262,2013-03-04 00:00:00,NaN,NaN,LAX,NYC,US,MNL,NYC,US,ORD,NYC,US,LHR,NYC,US,LHR,NYC,US,CCJJED,CCJJED,USUS,NaN,OSLISTISU,FK,V,Y,2013-03-10 18:20:00,2013-03-11 03:52:53,NaN,NaN,NaN,NULL
freq,75,372,402,59,59,12,84,NaN,NaN,30,42,389,20,36,334,30,47,394,27,35,352,23,44,365,11,11,270,NaN,6,122,80,897,6,6,NaN,NaN,NaN,861
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23935.433000,407.569000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.643000,NaN,NaN,NaN,NaN,NaN,NaN,0.532000,2013,3,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54459.101785,1490.087967,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.479355,NaN,NaN,NaN,NaN,NaN,NaN,1.976077,0,0,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,-8.000000,2013,3,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3493.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,-1.000000,2013,3,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6891.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,2013,3,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17940.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,2013,3,NaN


In [28]:
b.isnull()

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,...,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [87]:
b.isnull().sum()

act_date               0
source                 0
pos_ctry               0
pos_iata               0
pos_oid                0
rloc                   0
cre_date               0
duration               0
distance               0
dep_port               0
dep_city               0
dep_ctry               0
arr_port               0
arr_city               0
arr_ctry               0
lst_port               0
lst_city               0
lst_ctry               0
brd_port               0
brd_city               0
brd_ctry               0
off_port               0
off_city               0
off_ctry               0
mkt_port               0
mkt_city               0
mkt_ctry               0
intl                   0
route                  0
carrier                0
bkg_class              0
cab_class              0
brd_time               0
off_time               0
pax                    0
year                   0
month                  0
oid                    0
dtype: int64

In [60]:
non_null_counts = b.count()

In [61]:
non_null_counts

act_date               1000
source                 1000
pos_ctry               1000
pos_iata               1000
pos_oid                1000
rloc                   1000
cre_date               1000
duration               1000
distance               1000
dep_port               1000
dep_city               1000
dep_ctry               1000
arr_port               1000
arr_city               1000
arr_ctry               1000
lst_port               1000
lst_city               1000
lst_ctry               1000
brd_port               1000
brd_city               1000
brd_ctry               1000
off_port               1000
off_city               1000
off_ctry               1000
mkt_port               1000
mkt_city               1000
mkt_ctry               1000
intl                   1000
route                  1000
carrier                1000
bkg_class              1000
cab_class              1000
brd_time               1000
off_time               1000
pax                    1000
year                

In [67]:
#pd.set_option('display.max_columns', 100)
pd.set_option('display.max_columns', None)


In [68]:
b.head()

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,dep_city,dep_ctry,arr_port,arr_city,arr_ctry,lst_port,lst_city,lst_ctry,brd_port,brd_city,brd_ctry,off_port,off_city,off_ctry,mkt_port,mkt_city,mkt_ctry,intl,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
0,2013-03-05 00:00:00,1A,DE,a68dd7ae953c8acfb187a1af2dcbe123,1a11ae49fcbf545fd2afc1a24d88d2b7,ea65900e72d71f4626378e2ebd298267,2013-02-22 00:00:00,1708,0,ZRH,ZRH,CH,LHR,LON,GB,ZRH,ZRH,CH,LHR,LON,GB,ZRH,ZRH,CH,LHRZRH,LONZRH,CHGB,1,LHRZRH,VI,T,Y,2013-03-07 08:50:00,2013-03-07 11:33:37,-1,2013,3,NULL
1,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,SAL,SAL,SV,CLT,CLT,US,CLTSAL,CLTSAL,SVUS,1,SALATLCLT,NV,L,Y,2013-04-12 13:04:00,2013-04-12 22:05:40,1,2013,3,NULL
2,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,CLTSAL,CLTSAL,SVUS,1,CLTATLSAL,NV,U,Y,2013-07-15 07:00:00,2013-07-15 11:34:51,1,2013,3,NULL
3,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,AKL,AKL,NZ,SVO,MOW,RU,AKLSVO,AKLMOW,NZRU,1,AKLHKGSVO,XK,G,Y,2013-04-24 23:59:00,2013-04-25 16:06:31,1,2013,3,SYDA82546
4,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,AKLSVO,AKLMOW,NZRU,1,SVOHKGAKL,XK,G,Y,2013-05-14 20:15:00,2013-05-16 10:44:50,1,2013,3,SYDA82546


## 2) Select the columns of interest 

In [11]:
path='/home/dsc/Data/challenge/'
file_name='bookings.csv.bz2'
#file_name='bookings.sample.csv.bz2'
filepath=path+'/'+file_name

b=pd.read_csv(filepath, sep='^', usecols=['arr_port', 'pax','year'], nrows=100000)

In [12]:
b.head()

,arr_port,pax,year
0,LHR,-1,2013
1,CLT,1,2013
2,CLT,1,2013
3,SVO,1,2013
4,SVO,1,2013


## 3) What to do with NaN?



In [31]:
b.isnull().sum()

arr_port    0
pax         0
dtype: int64

In [32]:
b.count()

arr_port    1000
pax         1000
dtype: int64

In [94]:
b=b.dropna()

What do we have so far?

In [3]:
path='/home/dsc/Data/challenge/'
file_name='bookings.csv.bz2'
#file_name='bookings.sample.csv.bz2'
filepath=path+'/'+file_name

b=pd.read_csv(filepath, sep='^', usecols=['arr_port', 'pax','year'], nrows=100000)
b=b.dropna()

In [63]:
len(b.dropna())

1000

In [64]:
b.shape

(1000, 3)

In [65]:
b.head()

,arr_port,pax,year
0,LHR,-1,2013
1,CLT,1,2013
2,CLT,1,2013
3,SVO,1,2013
4,SVO,1,2013


## 4) make action plan
1) get only the bookings from 2013

2) group by arr_port

3) sort 

4) get top 10

#### 4.1) Get only the booking from 2013

In [4]:
b = b[b['year'] == 2013]
b.drop('year',axis=1, inplace=True)
b.head()

,arr_port,pax
0,LHR,-1
1,CLT,1
2,CLT,1
3,SVO,1
4,SVO,1


#### 4.2) group by arr_port

In [15]:
arr_ports=b.groupby('arr_port')

In [18]:
#arr_ports.get_group('ACC     ')

,arr_port,pax
118,ACC,3
119,ACC,3


In [72]:
for airport, val in arr_ports:
    print "%s" %(airport)
    print val['pax'].sum()

ACC     
6
ADD     
2
AGP     
6
ALA     
2
ALC     
4
ALG     
0
AMD     
1
AMS     
1
ANC     
0
ASU     
-10
ATH     
4
ATL     
3
ATQ     
1
AUH     
0
AUS     
2
AYT     
0
BBI     
0
BCN     
5
BEY     
4
BHM     
-1
BJV     
8
BJX     
2
BKK     
2
BLQ     
2
BLR     
0
BNA     
4
BNE     
6
BOG     
0
BOI     
3
BOM     
5
BOS     
11
BRD     
2
BRU     
2
BUD     
0
BUF     
2
BWI     
10
CAG     
0
CAI     
2
CAN     
0
CBR     
0
CCJ     
-3
CCS     
0
CDG     
4
CGK     
12
CHQ     
0
CLT     
5
CMB     
0
COK     
7
COS     
6
CTA     
0
CUN     
4
CUR     
0
CUZ     
0
CVG     
2
CZM     
4
DAB     
2
DAC     
4
DAY     
2
DCA     
50
DEL     
7
DEN     
6
DFW     
-2
DMM     
2
DOH     
3
DTW     
3
DXB     
0
ELP     
0
EWR     
8
EZE     
2
FAI     
4
FAO     
2
FCO     
3
FLL     
13
FRA     
4
FUE     
0
GAU     
-3
GIZ     
0
GRU     
0
GUM     
2
HAV     
0
HER     
4
HKG     
2
HMO     
1
HNL     
0
ICN     
0
IND     
3
ISB     
0
IST     
7
ISU     
0
IXE     
0

In [ ]:
for i, (airport, val) in enumerate(arr_ports):
    print "%s" %(airport)
    print val['pax'].sum()
    if (i==10) :break

ACC     
6
ADD     
2
AGP     
6
ALA     
2
ALC     
4
ALG     
0
AMD     
1
AMS     
1
ANC     
0
ASU     
-10
ATH     
4


In [17]:
arr_ports.sum().sort_values('pax', ascending=False).head()

,pax,year
arr_port,,
DCA,50,24156
JFK,14,32208
FLL,13,10065
PVG,13,28182
CGK,12,16104


In [36]:
topAirports = b.dropna().groupby(['arr_port']).sum().sort_values('pax', ascending=False).head(10)
#topAirports = b.dropna().groupby(['arr_port']).sum().sort_values('pax', ascending=False)['pax'].head(10)
topAirports

arr_port
DCA         50
JFK         14
FLL         13
PVG         13
CGK         12
LGA         12
SVO         12
LAS         11
BOS         11
MIA         11
Name: pax, dtype: int64

In [ ]:
import pandas as pd

path='/home/dsc/Data/challenge/'
file_name='bookings.csv.bz2'
#file_name='bookings.sample.csv.bz2'
filepath=path+'/'+file_name

b=pd.read_csv(filepath, sep='^', usecols=['arr_port', 'pax','year'], nrows=100000)
b=b.dropna()
b=b[b['year']==2013]
b.drop('year', axis=1, inplace=True)
arr_ports=b.groupby('arr_port')
topairports=arr_ports.sum().sort_values('pax', ascending=False).head(10)

In [ ]:
import pandas as pd

path='/home/dsc/Data/challenge/'
file_name='bookings.csv.bz2'
#file_name='bookings.sample.csv.bz2'
filepath=path+'/'+file_name

b=pd.read_csv(filepath, sep='^', usecols=['arr_port', 'pax','year'], nrows=100000)

topairports=b[b['year']==2013]\
            [['arr_port', 'pax']]\
            .dropna()\
            .groupby(['arr_port'])\
            .sum()\
            .sort_values('pax', ascending=False)\
            .head(10)


## 5) Adjust the code to work with Big data

In [31]:
##%%time
bc = pd.read_csv('bookings.sample.csv.bz2',sep='^',usecols=['arr_port','pax'], chunksize=4000)
for i,chunk in enumerate(bc):
    chunk_airports=chunk.dropna().groupby(['arr_port']).sum().reset_index().head()
    print "Chank Nummber:%d" % i
    print chunk_airports


Chank Nummber:0
   arr_port  pax
0  ABQ         4
1  ABV         0
2  ABZ        -2
3  ACC         0
4  ADD         4
Chank Nummber:1
   arr_port  pax
0  AAL         0
1  ABQ         8
2  ABV         2
3  ABZ        -4
4  ACC         2
Chank Nummber:2
   arr_port  pax
0  ACC         2
1  ADZ         2
2  AEP         2
3  AFA         2
4  AJA         4


In [ ]:
%%time
import pandas as pd

path='/home/dsc/Data/challenge'
#file_name='bookings.csv.bz2'
file_name='bookings.sample.csv.bz2'
filepath=path+'/'+file_name

bc=pd.read_csv(filepath, sep='^', usecols=['arr_port', 'pax','year'], chunksize=8000)

all_chunks=pd.DataFrame()

for i, chunk in enumerate(bc):
    print "Chunk number %d has size :%d" %(i, len(chunk))
    b=chunk.dropna() 
    b=b[b['year']==2013]
    b.drop('year', axis=1, inplace=True)
    arr_ports=b.groupby('arr_port').sum().reset_index()
    all_chunks=all_chunks.append(arr_ports)
#all_chunks=pd.concat([all_chunks,arr_ports])

    
allairports=all_chunks.groupby('arr_port')\
            .pax.sum().reset_index().sort_values('pax', ascending=False)

topairports=allairports.head(10)

### Now we've got top airports

we have to install geobases
pip install GeoBases

In [80]:
from GeoBases import GeoBase

In [81]:
geoDict = GeoBase(data='ori_por', verbose=False)
#geoDict = GeoBase(data='ori_por')

In [82]:
type(geoDict)

GeoBases.GeoBaseModule.GeoBase

In [83]:
geoDict.keys()[:10]

['MLC',
 'XMZ',
 'XQP@1',
 'JED@1',
 'MFI@1',
 'JGO@1',
 'YAL@1',
 'NDP',
 'GEG@1',
 'NDS']

In [84]:
geoDict.fields

['__key__',
 '__dup__',
 '__par__',
 '__lno__',
 'iata_code',
 'icao_code',
 'faa_code',
 'is_geonames',
 'geoname_id',
 'valid_id',
 'name',
 'asciiname',
 'lat',
 'lng',
 'fclass',
 'fcode',
 'page_rank',
 'date_from',
 'date_until',
 'comment',
 'country_code',
 'cc2',
 'country_name',
 'continent_name',
 'adm1_code',
 'adm1_name_utf',
 'adm1_name_ascii',
 'adm2_code',
 'adm2_name_utf',
 'adm2_name_ascii',
 'adm3_code',
 'adm4_code',
 'population',
 'elevation',
 'gtopo30',
 'timezone',
 'gmt_offset',
 'dst_offset',
 'raw_offset',
 'moddate',
 'city_code',
 'city_name_utf',
 'city_name_ascii',
 'tvl_por_list@raw',
 'tvl_por_list',
 'state_code',
 'location_type@raw',
 'location_type',
 'wiki_link',
 'alt_name_section@raw',
 'alt_name_section',
 '__gar__']

In [302]:
geoDict.get?

In [85]:
geoDict.get('LHR','city_name_utf')

'London'

In [86]:
geoDict.get('LHR')['name']

'London Heathrow Airport'

In [87]:
topairports.arr_port[0]

'HKG     '

### How do we remove white spaces?

In [88]:
topairports.arr_port[0].strip()

'HKG'

### How do we put the name of each airport into the DF?

In [89]:
topairports['Airport Name']=topairports.arr_port.map(lambda x: geoDict.get(x.strip())['name'])

In [90]:
allairports['arr_port']=allairports.arr_port.map(lambda x: x.strip())
allairports['Airport Name']=allairports.arr_port.map(lambda x: geoDict.get(x)['name'])

,arr_port,pax,AirportName
0,HKG,112,Hong Kong International Airport
1,LGA,95,Hong Kong International Airport
2,ORD,94,Hong Kong International Airport
3,JFK,92,Hong Kong International Airport
4,SFO,91,Hong Kong International Airport
5,LAX,91,Hong Kong International Airport
6,MCO,90,Hong Kong International Airport
7,DCA,82,Hong Kong International Airport
8,DEN,79,Hong Kong International Airport
9,LHR,76,Hong Kong International Airport


In [ ]:
allairports.to_csv('all_airports.csv', sep='^', index=False)

In [ ]:
all_air=pd.read_csv('all_airports.csv', sep='^')
all_air.head()

In [ ]:

#allairports.to_csv('all_airports.csv', sep='^')
#all_air=pd.read_csv('all_airports.csv', sep='^')
#all_air.head()